In [1]:
import pandas as pd
import numpy as np
from scapy.all import rdpcap, TCP
import os
import glob
from datetime import datetime, timedelta
from pathlib import Path
import sys

sys.path.append(r'C:\Users\Echo\Desktop\modbus-detection\src')
import config

print("✓ 开始标签诊断")
print(f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✓ 开始标签诊断
当前时间: 2026-01-08 09:13:12


In [2]:
import pickle

with open(config.DATA_PROCESSED / 'stage1_results.pkl', 'rb') as f:
    stage1 = pickle.load(f)

print(f"External标签文件: {len(stage1['external_labels'])} 个")
print(f"IED标签文件: {len(stage1['ied_labels'])} 个")
print(f"SCADA标签文件: {len(stage1['scada_labels'])} 个")

External标签文件: 6 个
IED标签文件: 7 个
SCADA标签文件: 21 个


In [3]:
print("=" * 60)
print("1. External Attack CSV标签检查")
print("=" * 60)

# 读取第一个CSV
external_csv = stage1['external_labels'][0]
print(f"\n检查文件: {os.path.basename(external_csv)}")

df_ext = pd.read_csv(external_csv)

print(f"\n列名: {list(df_ext.columns)}")
print(f"总行数: {len(df_ext)}")

print("\n前5行数据:")
print(df_ext.head())

print("\n时间戳示例（前10条）:")
for i, ts in enumerate(df_ext['Timestamp'].head(10)):
    print(f"  {i+1}. {ts} (类型: {type(ts)})")

print("\n检查是否有TransactionID列:")
if 'TransactionID' in df_ext.columns:
    print("  ✓ 有TransactionID列")
    print(f"  示例: {df_ext['TransactionID'].head(3).tolist()}")
else:
    print("  ✗ 没有TransactionID列")

print("\n检查是否有TargetIP列:")
if 'TargetIP' in df_ext.columns:
    print("  ✓ 有TargetIP列")
    print(f"  示例: {df_ext['TargetIP'].head(3).tolist()}")
else:
    print("  ✗ 没有TargetIP列")

print("\n攻击类型分布:")
print(df_ext['Attack'].value_counts())

1. External Attack CSV标签检查

检查文件: 01-01-2023-1.csv

列名: ['Timestamp', 'Attack']
总行数: 1

前5行数据:
                 Timestamp               Attack
0  2023-01-01 21:00:44.389  Recon. Range: 65535

时间戳示例（前10条）:
  1. 2023-01-01 21:00:44.389 (类型: <class 'str'>)

检查是否有TransactionID列:
  ✗ 没有TransactionID列

检查是否有TargetIP列:
  ✗ 没有TargetIP列

攻击类型分布:
Attack
Recon. Range: 65535    1
Name: count, dtype: int64


In [4]:
print("\n" + "=" * 60)
print("2. External时间戳解析测试")
print("=" * 60)

# 尝试不同的解析方法
print("\n方法1: 使用pd.to_datetime默认格式")
try:
    ts_parsed = pd.to_datetime(df_ext['Timestamp'])
    print(f"  ✓ 成功解析 {len(ts_parsed)} 条")
    print(f"  时间范围: {ts_parsed.min()} 到 {ts_parsed.max()}")
except Exception as e:
    print(f"  ✗ 失败: {e}")

print("\n方法2: 使用format='mixed'")
try:
    ts_parsed = pd.to_datetime(df_ext['Timestamp'], format='mixed')
    print(f"  ✓ 成功解析 {len(ts_parsed)} 条")
    print(f"  时间范围: {ts_parsed.min()} 到 {ts_parsed.max()}")
except Exception as e:
    print(f"  ✗ 失败: {e}")

print("\n方法3: 使用errors='coerce'")
try:
    ts_parsed = pd.to_datetime(df_ext['Timestamp'], errors='coerce')
    invalid_count = ts_parsed.isna().sum()
    valid_count = len(ts_parsed) - invalid_count
    print(f"  ✓ 成功: {valid_count} 条有效, {invalid_count} 条无效")
    if valid_count > 0:
        print(f"  时间范围: {ts_parsed.min()} 到 {ts_parsed.max()}")
    if invalid_count > 0:
        print(f"\n  无效时间戳示例:")
        invalid_indices = df_ext[ts_parsed.isna()].index[:5]
        for idx in invalid_indices:
            print(f"    行{idx}: {df_ext.loc[idx, 'Timestamp']}")
except Exception as e:
    print(f"  ✗ 失败: {e}")


2. External时间戳解析测试

方法1: 使用pd.to_datetime默认格式
  ✓ 成功解析 1 条
  时间范围: 2023-01-01 21:00:44.389000 到 2023-01-01 21:00:44.389000

方法2: 使用format='mixed'
  ✓ 成功解析 1 条
  时间范围: 2023-01-01 21:00:44.389000 到 2023-01-01 21:00:44.389000

方法3: 使用errors='coerce'
  ✓ 成功: 1 条有效, 0 条无效
  时间范围: 2023-01-01 21:00:44.389000 到 2023-01-01 21:00:44.389000


In [5]:
print("\n" + "=" * 60)
print("3. SCADA CSV标签检查")
print("=" * 60)

# 读取第一个SCADA CSV
scada_csv = stage1['scada_labels'][0]
print(f"\n检查文件: {os.path.basename(scada_csv)}")

df_scada = pd.read_csv(scada_csv)

print(f"\n列名: {list(df_scada.columns)}")
print(f"总行数: {len(df_scada)}")

print("\n前5行数据:")
print(df_scada.head())

print("\n时间戳示例（前10条）:")
for i, ts in enumerate(df_scada['Timestamp'].head(10)):
    print(f"  {i+1}. {ts}")

# 找出问题时间戳
print("\n" + "=" * 60)
print("查找异常时间戳")
print("=" * 60)

try:
    ts_parsed = pd.to_datetime(df_scada['Timestamp'], errors='coerce')
    invalid_count = ts_parsed.isna().sum()
    
    print(f"有效: {len(ts_parsed) - invalid_count} 条")
    print(f"无效: {invalid_count} 条")
    
    if invalid_count > 0:
        print(f"\n无效时间戳示例（前20条）:")
        invalid_indices = df_scada[ts_parsed.isna()].index[:20]
        for idx in invalid_indices:
            print(f"  行{idx}: {df_scada.loc[idx, 'Timestamp']}")
        
        # 统计无效时间戳的模式
        invalid_timestamps = df_scada.loc[ts_parsed.isna(), 'Timestamp']
        print(f"\n无效时间戳的模式:")
        print(invalid_timestamps.value_counts().head(10))
except Exception as e:
    print(f"解析失败: {e}")


3. SCADA CSV标签检查

检查文件: 03-12-2023-1.csv

列名: ['Timestamp', 'TargetIP', 'Attack', 'TransactionID']
总行数: 107770

前5行数据:
                 Timestamp     TargetIP  \
0  2023-03-12 16:12:27.761  185.175.0.4   
1  2023-03-12 16:12:27.773  185.175.0.4   
2  2023-03-12 16:12:27.774  185.175.0.4   
3  2023-03-12 16:12:27.774  185.175.0.4   
4  2023-03-12 16:12:27.776  185.175.0.4   

                                      Attack  TransactionID  
0  Brute force or specific coil. Address: 13              5  
1         Brute force or specific - Complete          65535  
2  Brute force or specific coil. Address: 14              5  
3         Brute force or specific - Complete          65535  
4   Brute force or specific coil. Address: 8              5  

时间戳示例（前10条）:
  1. 2023-03-12 16:12:27.761
  2. 2023-03-12 16:12:27.773
  3. 2023-03-12 16:12:27.774
  4. 2023-03-12 16:12:27.774
  5. 2023-03-12 16:12:27.776
  6. 2023-03-12 16:12:27.776
  7. 2023-03-12 16:12:27.777
  8. 2023-03-12 16:12:27.777
  9

In [6]:
print("\n" + "=" * 60)
print("4. IED CSV标签检查")
print("=" * 60)

ied_csv = stage1['ied_labels'][0]
print(f"\n检查文件: {os.path.basename(ied_csv)}")

df_ied = pd.read_csv(ied_csv)

print(f"\n列名: {list(df_ied.columns)}")
print(f"总行数: {len(df_ied)}")

print("\n前5行数据:")
print(df_ied.head())

print("\n时间戳示例:")
for i, ts in enumerate(df_ied['Timestamp'].head(5)):
    print(f"  {i+1}. {ts}")

# 时间戳解析
try:
    ts_parsed = pd.to_datetime(df_ied['Timestamp'], errors='coerce')
    invalid_count = ts_parsed.isna().sum()
    print(f"\n有效: {len(ts_parsed) - invalid_count} 条")
    print(f"无效: {invalid_count} 条")
except Exception as e:
    print(f"解析失败: {e}")


4. IED CSV标签检查

检查文件: 03-23-2023-1.csv

列名: ['Timestamp', 'TargetIP', 'Attack', 'TransactionID']
总行数: 66

前5行数据:
                 Timestamp     TargetIP                        Attack  \
0  2023-03-23 05:09:22.829  185.175.0.2  Baseline Replay: In position   
1  2023-03-23 05:30:24.984  185.175.0.2  Baseline Replay: In position   
2  2023-03-23 05:51:26.274  185.175.0.2  Baseline Replay: In position   
3  2023-03-23 06:07:30.613  185.175.0.2  Baseline Replay: In position   
4  2023-03-23 06:35:34.772  185.175.0.2  Baseline Replay: In position   

   TransactionID  
0              1  
1           1238  
2           2479  
3           3424  
4           5077  

时间戳示例:
  1. 2023-03-23 05:09:22.829
  2. 2023-03-23 05:30:24.984
  3. 2023-03-23 05:51:26.274
  4. 2023-03-23 06:07:30.613
  5. 2023-03-23 06:35:34.772

有效: 66 条
无效: 0 条


In [7]:
print("\n" + "=" * 60)
print("5. PCAP时间戳检查")
print("=" * 60)

# External PCAP
print("\n5.1 External PCAP时间戳:")
external_pcaps = glob.glob(os.path.join(config.EXTERNAL_PCAP_DIR, "*.pcap"))
if external_pcaps:
    test_pcap = external_pcaps[0]
    print(f"检查文件: {os.path.basename(test_pcap)}")
    
    packets = rdpcap(test_pcap, count=100)
    modbus_packets = [p for p in packets if TCP in p and (p[TCP].sport == 502 or p[TCP].dport == 502)]
    
    if modbus_packets:
        print(f"\n前5个Modbus包的时间戳:")
        for i, pkt in enumerate(modbus_packets[:5]):
            ts_raw = datetime.fromtimestamp(float(pkt.time))
            ts_utc = ts_raw + timedelta(hours=3)
            print(f"  {i+1}. 原始: {ts_raw}")
            print(f"      UTC:  {ts_utc}")
            print()

# Benign PCAP
print("\n5.2 Benign PCAP时间戳:")
benign_pcaps = glob.glob(os.path.join(config.BENIGN_PCAP_DIR, "*.pcap"))
if benign_pcaps:
    test_pcap = benign_pcaps[0]
    print(f"检查文件: {os.path.basename(test_pcap)}")
    
    packets = rdpcap(test_pcap, count=100)
    modbus_packets = [p for p in packets if TCP in p and (p[TCP].sport == 502 or p[TCP].dport == 502)]
    
    if modbus_packets:
        print(f"\n前5个Modbus包的时间戳:")
        for i, pkt in enumerate(modbus_packets[:5]):
            ts = datetime.fromtimestamp(float(pkt.time))
            print(f"  {i+1}. {ts}")


5. PCAP时间戳检查

5.1 External PCAP时间戳:
检查文件: network-wide-normal-0.pcap

前5个Modbus包的时间戳:
  1. 原始: 2023-02-01 11:48:39.921403
      UTC:  2023-02-01 14:48:39.921403

  2. 原始: 2023-02-01 11:48:39.921463
      UTC:  2023-02-01 14:48:39.921463

  3. 原始: 2023-02-01 11:48:39.921484
      UTC:  2023-02-01 14:48:39.921484

  4. 原始: 2023-02-01 11:48:39.921816
      UTC:  2023-02-01 14:48:39.921816

  5. 原始: 2023-02-01 11:48:39.921843
      UTC:  2023-02-01 14:48:39.921843


5.2 Benign PCAP时间戳:
检查文件: network-wide-normal-14.pcap

前5个Modbus包的时间戳:
  1. 2023-01-24 04:52:05.777284
  2. 2023-01-24 04:52:05.777309
  3. 2023-01-24 04:52:05.777466
  4. 2023-01-24 04:52:05.777481
  5. 2023-01-24 04:52:05.778793


In [8]:
print("\n" + "=" * 60)
print("6. PCAP vs CSV 时间戳对比")
print("=" * 60)

# External对比
print("\n6.1 External Attack对比:")
if external_pcaps and len(stage1['external_labels']) > 0:
    # PCAP时间
    packets = rdpcap(external_pcaps[0], count=10)
    modbus_packets = [p for p in packets if TCP in p and (p[TCP].sport == 502 or p[TCP].dport == 502)]
    
    if modbus_packets:
        pcap_time_raw = datetime.fromtimestamp(float(modbus_packets[0].time))
        pcap_time_utc = pcap_time_raw + timedelta(hours=3)
        
        print(f"PCAP第一个包时间（原始）: {pcap_time_raw}")
        print(f"PCAP第一个包时间（+3h）:  {pcap_time_utc}")
        
        # CSV时间
        df_ext_csv = pd.read_csv(stage1['external_labels'][0])
        csv_time = pd.to_datetime(df_ext_csv['Timestamp'].iloc[0], errors='coerce')
        
        print(f"CSV第一条标签时间:        {csv_time}")
        
        # 计算时间差
        if not pd.isna(csv_time):
            diff_raw = abs((pcap_time_raw - csv_time).total_seconds())
            diff_utc = abs((pcap_time_utc - csv_time).total_seconds())
            
            print(f"\n时间差（原始）: {diff_raw:.1f} 秒")
            print(f"时间差（+3h）:  {diff_utc:.1f} 秒")
            
            if diff_utc < 60:
                print("✓ 时间戳基本对齐（时区转换+3h正确）")
            elif diff_raw < 60:
                print("⚠️  时区转换可能不需要（或方向错误）")
            else:
                print("✗ 时间戳差距过大，需要进一步调查")


6. PCAP vs CSV 时间戳对比

6.1 External Attack对比:
PCAP第一个包时间（原始）: 2023-02-01 11:48:39.921403
PCAP第一个包时间（+3h）:  2023-02-01 14:48:39.921403
CSV第一条标签时间:        2023-01-01 21:00:44.389000

时间差（原始）: 2645275.5 秒
时间差（+3h）:  2656075.5 秒
✗ 时间戳差距过大，需要进一步调查


In [9]:
print("\n" + "=" * 60)
print("7. 测试不同的匹配策略")
print("=" * 60)

# 简化版匹配测试
if external_pcaps and len(stage1['external_labels']) > 0:
    # 读取少量数据
    print("读取External数据（前1000个包）...")
    packets = rdpcap(external_pcaps[0], count=1000)
    
    # 解析PCAP
    pcap_data = []
    for pkt in packets:
        if TCP in pkt and (pkt[TCP].sport == 502 or pkt[TCP].dport == 502):
            try:
                payload = bytes(pkt[TCP].payload)
                if len(payload) >= 8:
                    ts = datetime.fromtimestamp(float(pkt.time)) + timedelta(hours=3)
                    txid = int.from_bytes(payload[0:2], byteorder='big')
                    dst_ip = pkt['IP'].dst
                    
                    pcap_data.append({
                        'timestamp': ts,
                        'dst_ip': dst_ip,
                        'txid': txid
                    })
            except:
                pass
    
    df_pcap_test = pd.DataFrame(pcap_data)
    print(f"✓ 解析了 {len(df_pcap_test)} 个Modbus包")
    
    # 读取CSV标签
    df_csv_test = pd.read_csv(stage1['external_labels'][0])
    df_csv_test['Timestamp'] = pd.to_datetime(df_csv_test['Timestamp'], errors='coerce')
    df_csv_test = df_csv_test.dropna(subset=['Timestamp'])
    
    print(f"✓ 读取了 {len(df_csv_test)} 条CSV标签")
    
    # 测试匹配
    print("\n测试匹配策略:")
    
    # 策略1: 仅时间窗口
    time_window = 1.0
    matched_time = 0
    for _, label in df_csv_test.iterrows():
        time_mask = abs((df_pcap_test['timestamp'] - label['Timestamp']).dt.total_seconds()) <= time_window
        if time_mask.sum() > 0:
            matched_time += 1
    
    print(f"  策略1（仅时间，±{time_window}s）: {matched_time}/{len(df_csv_test)} = {matched_time/len(df_csv_test)*100:.1f}%")
    
    # 策略2: 时间 + IP
    if 'TargetIP' in df_csv_test.columns:
        matched_time_ip = 0
        for _, label in df_csv_test.iterrows():
            time_mask = abs((df_pcap_test['timestamp'] - label['Timestamp']).dt.total_seconds()) <= time_window
            ip_mask = df_pcap_test['dst_ip'] == label['TargetIP']
            if (time_mask & ip_mask).sum() > 0:
                matched_time_ip += 1
        
        print(f"  策略2（时间+IP）: {matched_time_ip}/{len(df_csv_test)} = {matched_time_ip/len(df_csv_test)*100:.1f}%")
    
    # 策略3: 时间 + IP + TxID
    if 'TransactionID' in df_csv_test.columns and 'TargetIP' in df_csv_test.columns:
        matched_all = 0
        for _, label in df_csv_test.iterrows():
            time_mask = abs((df_pcap_test['timestamp'] - label['Timestamp']).dt.total_seconds()) <= time_window
            ip_mask = df_pcap_test['dst_ip'] == label['TargetIP']
            txid_mask = df_pcap_test['txid'] == label['TransactionID']
            if (time_mask & ip_mask & txid_mask).sum() > 0:
                matched_all += 1
        
        print(f"  策略3（时间+IP+TxID）: {matched_all}/{len(df_csv_test)} = {matched_all/len(df_csv_test)*100:.1f}%")
    
    # 尝试不同时间窗口
    print("\n测试不同时间窗口（仅时间匹配）:")
    for tw in [0.5, 1.0, 2.0, 5.0, 10.0]:
        matched = 0
        for _, label in df_csv_test.iterrows():
            time_mask = abs((df_pcap_test['timestamp'] - label['Timestamp']).dt.total_seconds()) <= tw
            if time_mask.sum() > 0:
                matched += 1
        print(f"  ±{tw}s: {matched}/{len(df_csv_test)} = {matched/len(df_csv_test)*100:.1f}%")


7. 测试不同的匹配策略
读取External数据（前1000个包）...
✓ 解析了 198 个Modbus包
✓ 读取了 1 条CSV标签

测试匹配策略:
  策略1（仅时间，±1.0s）: 0/1 = 0.0%

测试不同时间窗口（仅时间匹配）:
  ±0.5s: 0/1 = 0.0%
  ±1.0s: 0/1 = 0.0%
  ±2.0s: 0/1 = 0.0%
  ±5.0s: 0/1 = 0.0%
  ±10.0s: 0/1 = 0.0%


In [10]:
print("\n" + "=" * 60)
print("诊断总结")
print("=" * 60)

summary = f"""
1. CSV标签文件检查:
   - External: {len(stage1['external_labels'])} 个文件
   - IED: {len(stage1['ied_labels'])} 个文件
   - SCADA: {len(stage1['scada_labels'])} 个文件

2. 时间戳格式问题:
   - [需要根据上面的输出填写]

3. 匹配策略测试:
   - [需要根据上面的输出填写]

4. 建议的修复方案:
   - [将在下一步提供]
"""

print(summary)

print("\n诊断完成！")
print("请查看上面的输出，我们会根据结果制定修复方案。")


诊断总结

1. CSV标签文件检查:
   - External: 6 个文件
   - IED: 7 个文件
   - SCADA: 21 个文件

2. 时间戳格式问题:
   - [需要根据上面的输出填写]

3. 匹配策略测试:
   - [需要根据上面的输出填写]

4. 建议的修复方案:
   - [将在下一步提供]


诊断完成！
请查看上面的输出，我们会根据结果制定修复方案。


In [11]:
print("=" * 60)
print("验证External Attack的PCAP路径")
print("=" * 60)

# 当前配置的路径
print(f"\n当前External PCAP路径: {config.EXTERNAL_PCAP_DIR}")

# 检查该路径下的文件
external_pcaps = glob.glob(os.path.join(config.EXTERNAL_PCAP_DIR, "*.pcap"))
print(f"找到 {len(external_pcaps)} 个PCAP文件:")
for pcap in external_pcaps:
    print(f"  - {os.path.basename(pcap)}")

# 检查数据集根目录下的attack/external结构
print("\n查找真正的External Attack PCAP目录:")
external_base = os.path.join(config.DATASET_ROOT, "attack", "external")

print(f"\n扫描: {external_base}")
for root, dirs, files in os.walk(external_base):
    pcap_files = [f for f in files if f.endswith('.pcap')]
    if pcap_files:
        print(f"\n目录: {root}")
        print(f"PCAP文件数: {len(pcap_files)}")
        for pcap in pcap_files[:3]:
            print(f"  - {pcap}")

验证External Attack的PCAP路径

当前External PCAP路径: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\network-wide
找到 2 个PCAP文件:
  - network-wide-normal-0.pcap
  - network-wide-normal-1.pcap

查找真正的External Attack PCAP目录:

扫描: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\central-agent
PCAP文件数: 1
  - veth460b141-0.pcap

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\external-attacker\external-attacker-network-capture
PCAP文件数: 1
  - veth665f3cf-0.pcap

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\ied1a
PCAP文件数: 1
  - veth4edc015-0.pcap

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\ied1b
PCAP文件数: 1
  - vethd9e14c0-0.pcap

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\ied4c
PCAP文件数: 1
  - veth8bc3408-0.pcap

目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\net

In [12]:
print("=" * 60)
print("验证所有PCAP路径配置")
print("=" * 60)

# 1. IED路径
print("\n1. IED路径检查:")
print(f"当前配置: {config.IED_PCAP_DIR}")

ied_possible_paths = [
    os.path.join(config.DATASET_ROOT, "attack", "compromised-ied", "ied1b", "ied1b-network-captures"),
    os.path.join(config.DATASET_ROOT, "attack", "compromised-ied", "ied1b-network-captures"),
]

for path in ied_possible_paths:
    if os.path.exists(path):
        pcaps = glob.glob(os.path.join(path, "*.pcap"))
        print(f"  ✓ 找到: {path}")
        print(f"    PCAP文件数: {len(pcaps)}")
        if len(pcaps) > 0:
            for p in pcaps[:3]:
                print(f"      - {os.path.basename(p)}")
            break

# 2. SCADA路径
print("\n2. SCADA路径检查:")
print(f"当前配置: {config.SCADA_PCAP_DIR}")

scada_possible_paths = [
    os.path.join(config.DATASET_ROOT, "attack", "compromised-scada", "substation-wide-capture"),
    os.path.join(config.DATASET_ROOT, "attack", "compromised-scada", "network-wide"),
]

for path in scada_possible_paths:
    if os.path.exists(path):
        pcaps = glob.glob(os.path.join(path, "*.pcap"))
        print(f"  ✓ 找到: {path}")
        print(f"    PCAP文件数: {len(pcaps)}")
        if len(pcaps) > 0:
            for p in pcaps[:3]:
                print(f"      - {os.path.basename(p)}")
            break

# 3. 验证新的External路径
print("\n3. External新路径验证:")
new_external_path = os.path.join(config.DATASET_ROOT, "attack", "external", "external-attacker", "external-attacker-network-capture")
print(f"新路径: {new_external_path}")

if os.path.exists(new_external_path):
    pcaps = glob.glob(os.path.join(new_external_path, "*.pcap"))
    print(f"  ✓ 路径存在")
    print(f"    PCAP文件数: {len(pcaps)}")
    for p in pcaps:
        print(f"      - {os.path.basename(p)}")
else:
    print(f"  ✗ 路径不存在")

验证所有PCAP路径配置

1. IED路径检查:
当前配置: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\compromised-ied\ied1b\ied1b-network-captures
  ✓ 找到: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\compromised-ied\ied1b\ied1b-network-captures
    PCAP文件数: 6
      - vethc76bd3f-0.pcap
      - vethc76bd3f-1.pcap
      - vethc76bd3f-2.pcap

2. SCADA路径检查:
当前配置: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\compromised-scada\substation-wide-capture
  ✓ 找到: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\compromised-scada\substation-wide-capture
    PCAP文件数: 18
      - substation-0.pcap
      - substation-1.pcap
      - substation-10.pcap

3. External新路径验证:
新路径: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset\attack\external\external-attacker\external-attacker-network-capture
  ✓ 路径存在
    PCAP文件数: 1
      - veth665f3cf-0.pcap
